In [13]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
There's a new jupysql version available (0.10.2), you're running 0.10.0. To upgrade: pip install jupysql --upgrade


In [14]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


In [15]:
%%sql
create or replace table competencia_02 as 
select 
    *
from read_csv_auto('../datasets/competencia_02_crudo.csv')

In [ ]:
%%sql
create or replace table competencia_02 as 

with max_db as 
    (
        select max(foto_mes) as max_foto_mes_db
        from competencia_02
    )
select c.*,
    case
        when max_db.max_foto_mes_db = max_numero_de_cliente.max_foto_mes then 'CONTINUA'
        when CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
           (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) =  0 then 'BAJA+1'
        when CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
           (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) == 1 then 'BAJA+2'
         when CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
           (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) > 1 then 'CONTINUA'
    end as clase_ternaria
from competencia_02 c
    cross join max_db
    left join 
        (
            select numero_de_cliente, max(foto_mes) as max_foto_mes
            from competencia_02
            group by numero_de_cliente
        ) max_numero_de_cliente on max_numero_de_cliente.numero_de_cliente = c.numero_de_cliente 

,Success


In [ ]:
%%sql
COPY competencia_02 TO '../datasets/competencia_02.csv' (FORMAT CSV, HEADER)


UsageError: Cell magic `%%sql` not found.
